In [1]:
from sympy import *
from sympy.printing import latex

In [2]:
#Initialize symbols
x, y = symbols('x y', real=True)
r, s, t = symbols('r s t', real=True)
H = symbols('H', real=True, commutative=True, cls=Function)

#Define standard differentials
dx = lambda f : diff(f,x)
dy = lambda f : diff(f,y)
du = lambda f : diff(f,u)
dv = lambda f : diff(f,v)

Define Wirtinger derivatives $\partial = \frac12( \partial_x- i \partial_y)$ and $\bar \partial = \frac12(\partial_x + i \partial_y)$

In [4]:
wirtd = lambda f : (diff(f,x)-I*diff(f,y))/2
wirtdbar = lambda f : (diff(f,x)+I*diff(f,y))/2

Define twisted differentials
$\mathcal{D}_1F(z) = \partial F(z) -  \tfrac{\bar z}{2}F(z)$ and $\mathcal{D}_2F(z) = \bar \partial F(z) + \tfrac{z}{2}F(z)$ 

In [6]:
D1 = lambda f : (diff(f,x)-I*diff(f,y))/2-(x-I*y)/2*f
D2 = lambda f : (diff(f,x)+I*diff(f,y))/2+(x+I*y)/2*f
D1b = lambda f : (diff(f,x)+I*diff(f,y))/2-(x+I*y)/2*f
D2b = lambda f : (diff(f,x)-I*diff(f,y))/2+(x-I*y)/2*f

Formally, we need the twisted shift $\mathcal{T}_{w}F_0(z)= F_0(z-w)e^{i \mathrm{Im}(z \bar w)}$.
However, we can consider wlog only the case $w = 0$

In [8]:
Tw = lambda f : f 

Define $H$; here, the order of the Laguerre polynomial in $H$ can be changed. The code was tested for oders 1 to 5

In [10]:
#H = laguerre_poly(order,x**2+y**2)*exp(-(x**2+y**2)*1/2)
H = laguerre_poly(1,x**2+y**2)*exp(-(x**2+y**2)*1/2)
H

(-x**2 - y**2 + 1)*exp(-x**2/2 - y**2/2)

Calculate the covariance matrix between 

$(\xi(z), \xi'(z), \xi''(z)) := 
\bigg( F(z), \frac{\mathcal{D}_1F(z)}{(-\Delta H(0) + 1/2)^{1/2}}, \frac{\mathcal{D}_2F(z)}{(-\Delta H(0) - 1/2)^{1/2}} \bigg)$. 

and

$(\xi(0), \xi'(0), \xi''(0)) := 
\bigg( F(0), \frac{\mathcal{D}_1F(0)}{(-\Delta H(0) + 1/2)^{1/2}}, \frac{\mathcal{D}_2F(0)}{(-\Delta H(0) - 1/2)^{1/2}} \bigg)$. 


In [12]:
DeltaH = wirtd(wirtdbar(H))
DeltaH0 = DeltaH.subs([(x,0),(y,0)])

Covxi = zeros(3, 3)
Covxi[0,0] = Tw(H)
Covxi[1,0] = Tw(D1(H)/sqrt(-DeltaH0+1/S(2)))
Covxi[0,1] = -Tw(D1b(H)/sqrt(-DeltaH0+1/S(2)))
Covxi[2,0] = Tw(D2(H)/sqrt(-DeltaH0-1/S(2)))
Covxi[0,2] = -Tw(D2b(H)/sqrt(-DeltaH0-1/S(2)))
Covxi[1,1] = -Tw(D1(D1b(H))/(-DeltaH0+1/S(2)))
Covxi[1,2] = -Tw(D1(D2b(H))/sqrt(-DeltaH0+1/S(2))/sqrt(-DeltaH0-1/S(2)))
Covxi[2,1] = -Tw(D1b(D2(H))/sqrt(-DeltaH0+1/S(2))/sqrt(-DeltaH0-1/S(2)))
Covxi[2,2] = -Tw(D2(D2b(H))/(-DeltaH0-1/S(2)))

Covxi = Covxi.applyfunc(simplify)
Covxi

Matrix([
[                                         (-x**2 - y**2 + 1)*exp(-x**2/2 - y**2/2), sqrt(2)*(-x**3 - I*x**2*y - x*y**2 + 2*x - I*y**3 + 2*I*y)*exp(-x**2/2 - y**2/2)/2,                          (x - I*y)*exp(-x**2/2 - y**2/2)],
[sqrt(2)*(x**3 - I*x**2*y + x*y**2 - 2*x - I*y**3 + 2*I*y)*exp(-x**2/2 - y**2/2)/2,          (x**4 + 2*x**2*y**2 - 4*x**2 + y**4 - 4*y**2 + 2)*exp(-x**2/2 - y**2/2)/2, sqrt(2)*(-x**2 + 2*I*x*y + y**2)*exp(-x**2/2 - y**2/2)/2],
[                                                 (-x - I*y)*exp(-x**2/2 - y**2/2),                           sqrt(2)*(-x**2 - 2*I*x*y + y**2)*exp(-x**2/2 - y**2/2)/2,                                    exp(-x**2/2 - y**2/2)]])

Calculate the coefficients 

$c_{k,l} 
= \frac{1}{\pi^2}\int_{\mathbb{C}} \int_{\mathbb{C}} \bigl|(-\Delta H(0) + 1/2)|z|^2- (-\Delta H(0) - 1/2)|w|^2\bigr| L_k(|z|^2)\, L_l(|w|^2)  \,e^{-|z|^2-|w|^2} \,dA(z) dA(w)$

We substitute in the integral $|z|^2 = r$ and $|w^2|= s$ and thus get an additional factor of $\pi^2$, i.e.,

 $c_{k,l} 
= \int_{0}^{\infty} \int_{0}^{\infty} \bigl|(-\Delta H(0) + 1/2)r- (-\Delta H(0) - 1/2)s\bigr| L_k(r)\, L_l(s)  \,e^{-r-s} \,dr ds$

In [14]:
#Calculate coefficients
#substitute t = r-(-DeltaH0-1/S(2))/(-DeltaH0+1/S(2))*s
#i.e., r = (t+(-DeltaH0-1/S(2))/(-DeltaH0+1/S(2))*s)
subsxt = ([(r,(t+(-DeltaH0-1/S(2))/(-DeltaH0+1/S(2))*s))])
coeffs = zeros(3,3)
for ii in range(0,3):
  for jj in range(0,3):
    my_integrand = abs((-DeltaH0+1/S(2))*r-(-DeltaH0-1/S(2))*s)*laguerre_poly(ii,r)*laguerre_poly(jj,s)*exp(-r-s)
    coeffs[ii,jj] = integrate(my_integrand.subs(subsxt),(t,0,oo),(s,0,oo))
    coeffs[ii,jj] = coeffs[ii,jj] + integrate(my_integrand.subs(subsxt),(t,-(-DeltaH0-1/S(2))/(-DeltaH0+1/S(2))*s,0),(s,0,oo))
coeffs

Matrix([
[  5/3,   -1/9,  8/27],
[-14/9, -16/27,     0],
[ 8/27,  -8/27, -8/81]])

We define the expectations of Laguerre polynomials with Gaussian variables $\alpha, \beta, \gamma, \delta \in \{\xi(z), \xi'(z), \xi''(z), \xi(w), \xi'(w), \xi''(w)\}$ according to Proposition 8.1, i.e.,

$
\mathbb{E}[L_1(|\alpha|^2) L_1(|\beta|^2) L_1(|\gamma|^2) L_1(|\delta|^2)] = 
\Big|\mathbb{E}(\alpha \overline{\gamma})\mathbb{E}(\beta \overline{\delta}) + \mathbb{E}(\alpha \overline{\delta}) 
\mathbb{E}(\beta \overline{\gamma} )\Big|^2 
$

$ \mathbb{E} [L_1(|\alpha|^2) L_1(|\beta|^2) L_2(|\gamma|^2) ] = 2 \left| \mathbb{E}(\alpha \overline{\gamma}) \mathbb{E}(\beta \overline{\gamma}) \right|^2 $

$ \mathbb{E} [L_2(|\alpha|^2) L_2(|\gamma|^2 ) ] =  \left| \mathbb{E}(\alpha \overline{\gamma})\right|^4$

In [16]:
LagE1 = lambda n1,n2,n3,n4 : abs(Covxi[n1,n3]*Covxi[n2,n4]+Covxi[n1,n4]*Covxi[n2,n3])**2
LagE2 = lambda n1,n2,n3 : S(2)*abs(Covxi[n1,n3]*Covxi[n2,n3])**2
LagE3 = lambda n1,n2 : abs(Covxi[n1,n2])**4


We build the expectation $g(|z-w|^2) = \mathbb{E}[\phi(z)\phi(w)]$ where

$\phi(z) = c_{1,0} L_1(|\xi(z)|^2)L_1(|\xi'(z)|^2) + c_{0,1}L_1(|\xi(z)|^2)L_1(|\xi''(z)|^2) + c_{1,1}L_1(|\xi'(z)|^2)L_1(|\xi''(z)|^2) + c_{0,0} L_2(|\xi(z)|^2) + c_{2,0} L_2(|\xi'(z)|^2) + c_{0,2} L_2(|\xi''(z)|^2)$

by using the expectations of Laguerre polynomials and explicit coefficients $c_{k,l}$ above

In [18]:
EPhiPhi = 0
EPhiPhi = EPhiPhi + coeffs[0,0]**2 * LagE3(0,0) + coeffs[2,0]**2 * LagE3(1,1) + coeffs[0,2]**2 * LagE3(2,2)
EPhiPhi = EPhiPhi + coeffs[0,1]**2 * LagE1(0,2,0,2) + coeffs[1,0]**2 * LagE1(0,1,0,1) + coeffs[1,1]**2 * LagE1(1,2,1,2)
EPhiPhi = EPhiPhi + 2*coeffs[0,0]*(coeffs[0,1]*LagE2(0,2,0) + coeffs[1,0]* LagE2(0,1,0) + coeffs[1,1] * LagE2(1,2,0) + coeffs[2,0] * LagE3(0,1) + coeffs[0,2] * LagE3(0,2))
EPhiPhi = EPhiPhi + 2*coeffs[2,0]*(coeffs[0,1]*LagE2(0,2,1) + coeffs[1,0]* LagE2(0,1,1) + coeffs[1,1] * LagE2(1,2,1) + coeffs[0,2] * LagE3(1,2))
EPhiPhi = EPhiPhi + 2*coeffs[0,2]*(coeffs[0,1]*LagE2(0,2,2) + coeffs[1,0]* LagE2(0,1,2) + coeffs[1,1] * LagE2(1,2,2))
EPhiPhi = EPhiPhi + 2*coeffs[1,1]*(coeffs[0,1]*LagE1(0,2,1,2) + coeffs[1,0]* LagE1(0,1,1,2))
EPhiPhi = EPhiPhi + 2*coeffs[1,0]*(coeffs[0,1]*LagE1(0,2,0,1))

EPhiPhi = EPhiPhi.simplify()
EPhiPhi.subs([(y,0),(x**2,s)])

2*(2*s**8 - 116*s**7 + 2156*s**6 - 15040*s**5 + 48325*s**4 - 77836*s**3 + 62628*s**2 - 22110*s + 2091)*exp(-2*s)/729

Finally, we calculate $\int_{\mathbb{C}}g(|z|^2) \,dA(z)$

In [20]:
EPhiPhi = EPhiPhi.expand()
integrate(EPhiPhi,(x,-oo,oo),(y,-oo,oo))

7*pi/81